## Set up databse

In [1]:
user = "postgres"
password = "125808037"
dbname = "BusStopDatabase"
import psycopg2
from postgis import LineString, Point
from postgis import register
db = psycopg2.connect(dbname= dbname,user = user, password = password)
register(db)
cursor = db.cursor()
db.commit()

## Create tables

In [2]:
db.commit()
cursor.execute('CREATE TABLE IF NOT EXISTS bus ( \
                    "id" int primary key, \
                    "name" varchar (200) not null,\
                    "code" varchar (50) not null,\
                    "entreprise" varchar (200) not null,\
                    "operations_time" varchar (200) not null,\
                    "frequency" varchar (100) not null, \
                    "bus_count" int, \
                    "cost" int not null,\
                    "first_station" varchar(200) not null, \
                    "last_station" varchar (200) not null, \
                    "go_route_string" text not null,\
                    "re_route_string" text not null, \
                    "go_route" geometry(LineString) not null,\
                   "re_route" geometry(LineString) NOT NULL)')
cursor.execute('create table if not exists station (\
                    "id" int primary key,\
                    "name" varchar (200) not null, \
                    "fleetover" varchar(100), \
                    "geo" geometry(Point) not null )')
cursor.execute('create table if not exists bus_station (\
                "id" serial primary key, \
                "idbus" int references bus(id), \
                "idstation" int references station(id), \
                "direction" int not null, \
                "station_order" int not null \
                )')
db.commit()

## Config header

In [3]:
import requests
import time
import json

headers = {
    'Referer': 'http://timbus.vn/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 OPR/57.0.3098.116',
    'Origin': 'http://timbus.vn',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive',
}

## Get bus id 

In [4]:
bus = ["01 - Bến xe Gia Lâm - Bến xe Yên Nghĩa","02 - Bác Cổ - Bến xe Yên Nghĩa","03A - Bến xe Giáp Bát - Bến xe Gia Lâm","03B - Bến xe Nước Ngầm - Giang Biên (Long Biên)","04 - Long Biên - Bệnh viện Nội tiết trung ương cơ sở 2","05 - KĐT Linh Đàm 4 - Phú Diễn","06A - Bến xe Giáp Bát - Cầu Giẽ","06B - Bến xe Giáp Bát - Hồng Vân","06C - Bến xe Giáp Bát - Phú Minh","06D - Bến xe Giáp Bát - Tân Dân","06E - Bến xe Giáp Bát - Phú Túc","07 - Cầu Giấy - Nội Bài","08A - Long Biên - Đông Mỹ","08ACT - Long Biên - Đông Mỹ","08B - Long Biên - Vạn Phúc","08BCT - Long Biên - Vạn Phúc","09A - Bờ Hồ - Cầu Giấy - Khu Liên cơ quan Sở ngành HN","09ACT - Trần Khánh Dư - Khu liên cơ quan Sở ngành HN","09B - Bờ Hồ - Công viên Thống Nhất - Bến xe Mỹ Đình","09BCT - Trần Khánh Dư - Bến xe Mỹ Đình","100 - Long Biên - Khu đô thị Đặng Xá","101A - Bến xe Giáp Bát - Vân Đình","101B - Bến xe Giáp Bát - Đại Cường (Ứng Hòa)","102 - Bến xe Yên Nghĩa - Vân Đình","103A - Bến xe Mỹ Đình - Hương Sơn","103B - Bến xe Mỹ Đình - Hồng Quang - Hương Sơn","104 - Mỹ Đình - BX Nước Ngầm","105 - Đô Nghĩa - Cầu Giấy","106 - KĐT Mỗ Lao - TTTM Aeon Mall Long Biên","107 - Kim Mã - Làng văn hóa du lịch các dân tộc VN","108 - Bến xe Thường Tín - Minh Tân","109 - Mỹ Đình - Nội Bài","10A - Long Biên - Từ Sơn","10B - Long Biên - Trung Mầu","11 - Công viên Thống Nhất - Học Viện Nông Nghiệp","110 - BX Sơn Tây - Vườn Quốc gia Ba Vì - Đá Chông","111 - BX Sơn Tây - Bất Bạt","112 - Nam Thăng Long - Thạch Đà (Mê Linh)","113 - Đại Thắng (Phú Xuyên) - Bến đò Vườn Chuối","12 - Công viên Nghĩa Đô - Đại Áng","13 - CV nước Hồ Tây - Cổ Nhuế (HVCS)","14 - Bờ Hồ - Cổ Nhuế","14CT - Trần Khánh Dư - Cổ Nhuế","15 - Bến xe Gia Lâm - Phố Nỉ","16 - Bến xe Mỹ Đình - Bến xe Nước Ngầm","17 - Long Biên - Nội Bài","18 - ĐH Kinh tế QD - ĐH Kinh tế QD","19 - Trần Khánh Dư - KĐT Vinhomes Thăng Long","20A - Cầu Giấy - Bến xe Sơn Tây","20B - Nhổn - Bến xe Sơn Tây","21A - Bến xe Giáp Bát - Bến xe Yên Nghĩa","21B - Khu đô thị Pháp Vân - Bến xe Mỹ Đình","22A - Bến xe Gia Lâm - KĐT Trung Văn","22B - BX Mỹ Đình - KĐT Kiến Hưng","22C - Bến xe Giáp Bát - Khu đô thị Dương Nội","23 - Nguyễn Công Trứ - Nguyễn Công Trứ","24 - Long Biên - Cầu Giấy","25 - BV Nhiệt đới TW Cơ sở 2 - Bến xe Giáp Bát","26 - Mai Động - SVĐ Quốc gia","27 - Bến xe Yên Nghĩa - Bến xe Nam Thăng Long","28 - BX Nước Ngầm - Đông Ngạc - Đại học Mỏ","29 - BX Giáp Bát - Tân Lập","31 - Bách Khoa - Chèm (ĐH Mỏ)","32 - Bến xe Giáp Bát - Nhổn","33 - Cụm Công nghiệp Thanh Oai - Xuân Đỉnh","34 - Bến xe Mỹ Đình - Bến xe Gia Lâm","35A - Trần Khánh Dư - Nam Thăng Long","35B - Nam Thăng Long - Thanh Lâm","36 - Yên Phụ - Khu đô thị Linh Đàm","36CT - Yên Phụ - Khu đô thị Linh Đàm","37 - Bến xe Giáp Bát - Chương Mỹ","38 - BX Nam Thăng Long - Mai Động","39 - Công viên Nghĩa Đô - Bệnh viện Nội tiết TW CS II","40 - Công viên Thống Nhất - Văn Lâm","41 - Nghi Tàm - BX Giáp Bát","42 - Giáp Bát - Đức Giang","43 - Công viên Thống Nhất - Thị trấn Đông Anh","44 - Trần Khánh Dư - BX Mỹ Đình","45 - Time City - Bến xe Nam Thăng Long","46 - Bến xe Mỹ Đình - Thị trấn Đông Anh","47A - Long Biên - Bát Tràng","47B - Đại học Kinh tế Quốc dân - Kiêu Kỵ","48 - Bến xe Nước Ngầm - Chương Dương - Phúc Lợi (Long Biên)","49 - Trần Khánh Dư - Nhổn","50 - Long Biên - Khu đô thị Vân Canh","51 - Trần Khánh Dư - Trần Vỹ (Học vện Tư Pháp)","52A - Công viên Thống nhất - Lệ Chi","52B - Công viên Thống Nhất - Đặng Xá","53A - Hoàng Quốc Việt - Đông Anh","53B - Bến xe Mỹ Đình - Kim Hoa (Mê Linh)","54 - Long Biên - Bắc Ninh","55A - Khu đô thị Times City - Cầu Giấy","55B - TTTM Aeon Mall - Cầu Giấy","56A - Bến xe Nam Thăng Long - Núi Đôi","56B - Học viện Phật Giáo VN - Xuân Giang - Bắc Phú - Học viện Phật Giáo VN","57 - Nam Thăng Long - Khu công nghiệp Phú Nghĩa","58 - Long Biên - Thạch Đà","59 - TT Đông Anh - Học viện Nông nghiệp Việt Nam","60A - KĐT Pháp Vân, Tứ Hiệp - Công Viên Nước Hồ Tây","60B - Bến xe Nước Ngầm - BV Bệnh nhiệt đới TƯ (CS2)","61 - Vân Hà (Đông Anh) - CV Cầu Giấy","62 - Bến xe Yên Nghĩa - Bến xe Thường Tín","63 - Khu công nghiệp Bắc Thăng Long - Tiến Thịnh (Mê Linh)","64 - BX Mỹ Đình - Phố Nỉ - TTTM Bình An","65 - Thụy Lâm (Đông Anh) - Long Biên (ĐCT Long Biên)","68 - Hà Đông - Sân bay Nội Bài","72 - Bến xe Yên Nghĩa - Xuân Mai","74 - Bến xe Mỹ Đình - Xuân Khanh","84 - Cầu Diễn - Khu đô thị Linh Đàm","85 - Công viên Nghĩa Đô - Khu đô thị Thanh Hà","86 - Ga Hà Nội - Sân bay Nội Bài","86CT - Ga Hà Nội - Sân Bay Nội Bài","87 - Bến xe Mỹ Đình - Quốc Oai - Xuân Mai","88 - Bến xe Mỹ Đình - Hòa Lạc - Xuân Mai","89 - Bến xe Yên Nghĩa - Thạch Thất - Bến xe Sơn Tây","90 - Hào Nam - Cầu Nhật Tân - Sân bay Nội Bài","91 - Bến xe Yên Nghĩa - Kim Bài - Phú Túc","92 - Nhổn - Sơn Tây - Tây Đằng","93 - Nam Thăng Long - Bắc Sơn (Sóc Sơn)","94 - Bến xe Giáp Bát - Kim Bài","95 - Nam Thăng Long - Xuân Hòa","96 - Công viên Nghĩa Đô - Đông Anh","97 - Hoài Đức - Công viên Nghĩa Đô","98 - Yên Phụ - Gia Thụy - Aeon Mall Long Biên","99 - Kim Mã - Ngũ Hiệp (Thanh Trì)","BRT01 - Bến xe Yên Nghĩa - Kim Mã"]
bus_info = []
for i in range(len(bus)):
    data = [
        ('act','searchfull'),
        ('typ','1'),
        ('key',bus[i])
    ]
    response = requests.post('http://timbus.vn/Engine/Business/Search/action.ashx', headers=headers, data=data)
    bus_info.append(response.json()['dt'])
    

## Utils

In [5]:
def get_route(dt,direction):
    go_geo = dt[direction]['Geo']
    go_line = []
    for i in range(len(go_geo)):
        go_line.append((go_geo[i]['Lng'], go_geo[i]['Lat']))
    return go_line
def get_operations_time(dt):
    opt = dt['OperationsTime']
    time = opt.split('###')[0]
    return time

## Get bus and station geo infomation 

In [6]:
bus_geo = []
for i in range(len(bus_info)):
    data = [
        ('act', 'fleetdetail'),
        ('fid',str(bus_info[i]['Data'][0]['ObjectID'])),
    ]
    response = requests.post('http://timbus.vn/Engine/Business/Search/action.ashx', headers=headers, data=data)
    dt = response.json()['dt'] 
    bus_geo.append(response.json()['dt'])

## Insert bus information 

In [7]:
db.commit()
for i in range(len(bus_geo)):
    dt = bus_geo[i]
    if (dt['BusCount']!=''):
        cursor.execute('insert into bus(id,name,code,entreprise,operations_time, \
                    frequency, bus_count, cost, first_station, last_station, go_route_string,\
                    re_route_string, go_route, re_route) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)',\
                   [dt['FleetID'],dt['Name'],dt['Code'],dt['Enterprise'],dt['OperationsTime'], dt['Frequency'],\
                    dt['BusCount'],dt['CostInt'],dt['FirstStation'],dt['LastStation'], dt['Go']['Route'], \
                   dt['Re']['Route'],LineString(get_route(dt,'Go'),srid=4326), LineString(get_route(dt,'Re'),srid = 4326)])
        db.commit()
    else:
        cursor.execute('insert into bus(id,name,code,entreprise,operations_time, \
                    frequency, cost, first_station, last_station, go_route_string,\
                    re_route_string, go_route, re_route) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)',\
                   [dt['FleetID'],dt['Name'],dt['Code'],dt['Enterprise'],dt['OperationsTime'], dt['Frequency'],\
                    dt['CostInt'],dt['FirstStation'],dt['LastStation'], dt['Go']['Route'], \
                   dt['Re']['Route'],LineString(get_route(dt,'Go'),srid=4326), LineString(get_route(dt,'Re'),srid = 4326)])
        db.commit()

## Insert station information 

In [8]:
db.commit()
for i in range(len(bus_geo)):
    dt = bus_geo[i]
    for station in dt['Go']['Station']:
        cursor.execute('Insert into station(id,name,fleetover, geo) values (%s,%s,%s,%s) ON CONFLICT (id) DO NOTHING',\
                      (station['ObjectID'], station['Name'], station['FleetOver'], \
                       Point(x=float(station['Geo']['Lng']), y = float(station['Geo']['Lat']), srid=4326)))
        db.commit()
    for station in dt['Re']['Station']:
        cursor.execute('Insert into station(id,name, fleetover, geo) values (%s,%s,%s,%s) ON CONFLICT (id) DO NOTHING',\
                      (station['ObjectID'], station['Name'], station['FleetOver'], \
                       Point(x=float(station['Geo']['Lng']), y = float(station['Geo']['Lat']), srid=4326)))
        db.commit()

## Insert relation between bus and station

Go = 0, Re = 1

In [9]:
db.commit()
for i in range(len(bus_geo)):
    dt = bus_geo[i]
    go_station = dt['Go']['Station']
    re_station = dt['Re']['Station']
    for j in range(len(go_station)):
        cursor.execute('Insert into bus_station(idbus, idstation, direction, station_order) values (%s,%s,%s,%s)',\
                      (dt['FleetID'], go_station[j]['ObjectID'], 0, j))
        db.commit()
    for j in range(len(re_station)):
        cursor.execute('Insert into bus_station(idbus, idstation, direction, station_order) values (%s,%s,%s,%s)',\
                      (dt['FleetID'], re_station[j]['ObjectID'], 1, j))
        db.commit()

# Test

In [7]:
bus_geo[1]

{'FleetID': 2,
 'Enterprise': 'Xí nghiệp Xe buýt nhanh BRT Hà Nội',
 'Code': '02',
 'Name': 'Bác Cổ - Bến xe Yên Nghĩa',
 'OperationsTime': '0|T2 - T6|05:00,22:30;0|T7|05:00,22:30;0|CN|05:02,22:30;###1|T2 - T6|05:00,22:30;1|T7|05:00,22:30;1|CN|05:00,22:31;',
 'Frequency': '5 -10- 15 phút/chuyến',
 'BusCount': '28',
 'Cost': '7000VNĐ/lượt',
 'CostInt': 7000,
 'FirstStation': 'Bác Cổ',
 'LastStation': 'Bến xe Yên Nghĩa',
 'Go': {'Anomaly': 0,
  'Route': 'Bác Cổ - Trần Khánh Dư - Trần Hưng Đạo - Lê Thánh Tông - Hai Bà Trưng - Quang Trung - Tràng Thi - Điện Biên Phủ - Trần Phú - Chu Văn An - Tôn Đức Thắng - Nguyễn Lương Bằng - Tây Sơn - Ngã tư Sở - Nguyễn Trãi - Trần Phú (Hà Đông) - Quang Trung (Hà Đông) - Ba La - Quốc lộ 6 - Bến xe Yên Nghĩa',
  'Geo': [{'Lng': 105.86083333333295, 'Lat': 21.0231833333333},
   {'Lng': 105.86086617625517, 'Lat': 21.02296871623681},
   {'Lng': 105.86112700694366, 'Lat': 21.022213303854727},
   {'Lng': 105.86132082672088, 'Lat': 21.021706564414863},
   {'Lng'